In [15]:
import numpy as np
import pandas as pd
import datetime
import math
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.externals import joblib
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, precision_score, recall_score, confusion_matrix, auc, roc_curve
from sklearn.model_selection import train_test_split
import random

In [2]:
df = pd.read_csv("モデル構築用.csv", encoding="sjis")
origin_columns = df.columns
df.columns = ["offer_date","NO","offer_time","baitai_dai","baitai_shousai","uketsuke_jikantai","uketsuke_keitai","shinki_jidokeiyaku","toriatsukai_tenban","kanri_tenban","sex","age","birth","jis","doku","sinshoku_dai","sinshoku_shousai","koyo","shugyo","kaisha_kibo","nenshu","zaisha","jukyo","kyoju","yachin","kazoku_kousei","hoken_shokushu","shunyu_shoumei","dokushin","doukyo","fuyo","fuyo_child","shotoku_shoumei","meigi","jigyo_keitai","jigyosho","month12_jiko"]
df.head()

,offer_date,NO,offer_time,baitai_dai,baitai_shousai,uketsuke_jikantai,uketsuke_keitai,shinki_jidokeiyaku,toriatsukai_tenban,kanri_tenban,...,shunyu_shoumei,dokushin,doukyo,fuyo,fuyo_child,shotoku_shoumei,meigi,jigyo_keitai,jigyosho,month12_jiko
0,2013/2/3,1.820570e+12,133504,3,2,3,502,0,182,182,...,0,1,2,0,0,0,0,6,0,0
1,2013/6/18,5.050290e+12,145329,5,99,0,1,2814,2814,505,...,0,0,1,2,1,0,0,0,0,0
2,2013/6/28,1.820590e+12,115136,5,99,0,1,2779,2779,182,...,1,0,1,1,0,8,1,0,0,0
3,2013/8/9,1.270660e+12,95034,5,99,0,1,223,223,127,...,0,1,2,0,0,0,0,0,0,0
4,2014/1/27,2.090350e+12,165240,5,99,0,1,1836,1836,209,...,0,0,2,0,0,0,0,0,0,0


In [3]:
df = df.drop(['offer_date', 'birth'], axis=1)
df.head()

,NO,offer_time,baitai_dai,baitai_shousai,uketsuke_jikantai,uketsuke_keitai,shinki_jidokeiyaku,toriatsukai_tenban,kanri_tenban,sex,...,shunyu_shoumei,dokushin,doukyo,fuyo,fuyo_child,shotoku_shoumei,meigi,jigyo_keitai,jigyosho,month12_jiko
0,1.820570e+12,133504,3,2,3,502,0,182,182,1,...,0,1,2,0,0,0,0,6,0,0
1,5.050290e+12,145329,5,99,0,1,2814,2814,505,1,...,0,0,1,2,1,0,0,0,0,0
2,1.820590e+12,115136,5,99,0,1,2779,2779,182,1,...,1,0,1,1,0,8,1,0,0,0
3,1.270660e+12,95034,5,99,0,1,223,223,127,1,...,0,1,2,0,0,0,0,0,0,0
4,2.090350e+12,165240,5,99,0,1,1836,1836,209,1,...,0,0,2,0,0,0,0,0,0,0


In [4]:
#trainとtestに分割
train_df, test_df = train_test_split(df, test_size = 0.1, random_state=0)
train_X = train_df.drop('month12_jiko',axis=1)
test_X = test_df.drop('month12_jiko',axis=1)
train_y = train_df['month12_jiko']
test_y = test_df['month12_jiko']

In [61]:
np.sum(train_df['month12_jiko']==1)

2908

In [26]:
np.array(train_df)

array([[5.72064e+12, 1.20956e+05, 4.00000e+00, ..., 0.00000e+00,
        0.00000e+00, 1.00000e+00],
       [3.20073e+12, 2.03034e+05, 1.00000e+00, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00],
       [3.17070e+12, 1.63012e+05, 1.00000e+00, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00],
       ...,
       [3.20074e+12, 1.91818e+05, 5.00000e+00, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00],
       [2.63113e+12, 1.93449e+05, 5.00000e+00, ..., 0.00000e+00,
        0.00000e+00, 1.00000e+00],
       [1.30632e+11, 1.01822e+05, 1.00000e+00, ..., 0.00000e+00,
        0.00000e+00, 1.00000e+00]])

In [47]:
hoge_list = random.sample(list(np.array(train_df)), np.array(train_df).shape[0])

In [48]:
len(hoge_list)

9565

In [49]:
np.sum(test_df['month12_jiko']==1)

150

In [63]:
count = 0
hoge_list = random.sample(list(np.array(train_df)), np.array(train_df).shape[0])
equal_train_list = []
for i in range(np.array(train_df).shape[0]):
    if hoge_list[i][34] == 1:
        equal_list.append(hoge_list[i])
        count += 1
    if count == np.sum(train_df['month12_jiko']==1):
        break
count = 0
for i in range(np.array(train_df).shape[0]):
    if hoge_list[i][34] == 0:
        equal_list.append(hoge_list[i])
        count += 1
    if count == np.sum(train_df['month12_jiko']==1):
        break
equal_train_list = random.sample(equal_train_list, len(equal_train_list))

In [ ]:
count = 0
hoge_list = random.sample(list(np.array(test_df)), np.array(test_df).shape[0])
equal_test_list = []
for i in range(np.array(train_df).shape[0]):
    if hoge_list[i][34] == 1:
        equal_list.append(hoge_list[i])
        count += 1
    if count == np.sum(train_df['month12_jiko']==1):
        break
count = 0
for i in range(np.array(train_df).shape[0]):
    if hoge_list[i][34] == 0:
        equal_list.append(hoge_list[i])
        count += 1
    if count == np.sum(train_df['month12_jiko']==1):
        break
equal_train_list = random.sample(equal_train_list, len(equal_train_list))